In [ ]:
import pandas as pd
import numpy as np

In [ ]:
results = pd.read_csv("data/results.csv")

In [ ]:
results 

In [ ]:
athletes = pd.read_csv(filepath_or_buffer="data/athletes.txt", sep=";")

athletes

In [ ]:
merged = pd.merge(left = athletes, right = results, how="inner")

In [ ]:
# Setting index to name and born since this an unique way to identify
merged.set_index(["name","born"]).head()

In [ ]:
merged["medal"].describe()

In [ ]:

merged.groupby('name')['medal'].sum()


### Lösungsvariante 1 

In [ ]:
gold_medals_df = merged[merged['medal'] == 'Gold']
gold_medals_count = gold_medals_df['name'].value_counts()
print(gold_medals_count)

# Exercise 1 aufgabe 2

In [ ]:
since2012 = merged[merged['year'] >= 2012]
since2012.nunique()

# Exerscice 1 Aufgabe 3

1. Count se Sex == Female 
2. descending ( lowest country ) = true mäßig ( vllt reindex oder so )
3. Iwie barchart lol 
 

In [ ]:
female = merged[merged['sex'] == "Female"]

sexAndCountry = merged[["sex", "country"]]
female = sexAndCountry[sexAndCountry['sex'] == "Female"]
male = sexAndCountry[sexAndCountry['sex'] == "Male"]
beide = merged[["sex", "country"]]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame named 'merged' with relevant data
# Replace 'merged' with your actual DataFrame name

# Calculate the ratio of female participants for each country
merged['Female Ratio'] = merged['sex']=="Female".count() / (merged['Female Participants'] + merged['Male Participants'])

# Group by country and calculate the mean ratio for each country
country_female_ratio = merged.groupby('Country')['Female Ratio'].mean()

# Sort the countries based on the ratio in ascending order and select the lowest 10
lowest_female_ratio_countries = country_female_ratio.sort_values().head(10)

# Create a bar chart
plt.figure(figsize=(12, 6))
lowest_female_ratio_countries.plot(kind='bar', color='skyblue')
plt.title('Top 10 Countries with Lowest Ratio of Female Participants')
plt.xlabel('Country')
plt.ylabel('Female Ratio')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
male_participants_count = merged[merged['sex'] == 'Male']
female_participants_count = merged[merged['sex'] == 'Female']

merged['Female Ratio'] = merged['sex'].apply(lambda x: 1 if x == 'Female' else 0)
male_participants_count = merged[merged['sex'] == 'Male'].shape[0]

print("Number of Male Participants:", male_participants_count)




In [ ]:
countries = merged["country"].unique()

In [ ]:
#Was brauchen wir? 
#Wie viele Frauen und Männer pro Country gibt 
perCountry = merged.groupby(['country', 'sex']).size().unstack(fill_value=0)
#Den Frauenanteil berechnen 
perCountry['Female Percentage'] = (perCountry['Female'] / (perCountry['Female'] + perCountry['Male'])) * 100

#perCountry["Fraunanteil"] = perCountry["Female"] + perCountry["Male"]
#Die letzen 10 ausgeben 

perCountry = perCountry.sort_values("Female Percentage").head(10)


plt.figure(figsize=(12, 6))
perCountry[["Female Percentage"]].plot(kind='bar', color='skyblue')
plt.title('Top 10 Countries with Lowest Ratio of Female Participants')
plt.xlabel('Country')
plt.ylabel('Female Ratio')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Exploring Data
What are the three most common firstnames of female athletes from the country ‘Republic of
Korea’?

In [ ]:

# Filter for female athletes from 'Republic of Korea'
female_korean_athletes = merged[(merged['sex'] == 'Female') & (merged['country'] == 'Republic of Korea')]

# Extract and isolate the first names
female_korean_athletes['First Name'] = female_korean_athletes['name'].apply(lambda x: x.split('•')[0])

# Count the occurrences of each first name
common_first_names = female_korean_athletes['First Name'].value_counts().head(3)

print("The three most common first names of female athletes from Republic of Korea:")
print(common_first_names)

In [ ]:
female_korean_athletes = merged[(merged['sex'] == 'Female')]

female_korean_athletes.head()

In [ ]:
female_korean_athletes = merged[(merged['sex'] == 'Female') & (merged['country'] == 'Republic of Korea')]

# Remove duplicates based on the athlete's name
female_korean_athletes_unique = female_korean_athletes.drop_duplicates(subset=['name'])

# Preprocess the 'athlete' column and extract the first names
female_korean_athletes_unique['First Name'] = female_korean_athletes_unique['name'].apply(lambda x: str(x).split('•')[0] if pd.notnull(x) else None)

# Drop rows with missing first names
female_korean_athletes_unique.dropna(subset=['First Name'], inplace=True)

# Count the occurrences of each first name
common_first_names = female_korean_athletes_unique['First Name'].value_counts().head(3)

print("The three most common first names of female athletes from Republic of Korea:")
print(common_first_names)

Who is the tallest female athlete? Provide the name, height, country and sport.

In [ ]:
female_athletes = merged[(merged['sex'] == 'Female')]
female_athletes = female_athletes.dropna(subset=['measurements'])

print(female_athletes["measurements"].unique())




In [ ]:
female_athletes = merged[(merged['sex'] == 'Female')]
female_athletes = female_athletes.dropna(subset=['measurements'])  

# Filter rows containing 'cm' in the 'measurements' column
female_athletes = female_athletes[female_athletes['measurements'].str.contains('cm')]

# Extract and convert height to integer
female_athletes['height'] = female_athletes['measurements'].str.extract('(\d+)').astype(int)

# Sort the DataFrame by height
reduced = female_athletes[["name", "height", "country", "sport"]].sort_values("height")

# Display the last 20 rows
print(reduced.head(25))


# aufgabe 5 mit api rumspielen

In [ ]:
import requests
import pandas as pd

# Make the API request
# SP.POP.TOTL for the indicator total propulation
# format = json to get a json file, the defaut is xml wich would need a external package to process
# date = 2016 to limit the returns only to the year in question wich is 2016
# per_page = 5000 to list all entrys in a single api request

url = "http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json&date=2016&per_page=5000"
response = requests.get(url)

data = response.json()[1]

In [ ]:
transformed_data = []

# Iterate through the data and extract the relevant fields
for entry in data:
    transformed_entry = {
        'Indicator ID': entry['indicator']['id'],
        'Indicator Value': entry['indicator']['value'],
        'Country ID': entry['country']['id'],
        'Country Value': entry['country']['value'],
        'Country ISO3 Code': entry['countryiso3code'],
        'Date': entry['date'],
        'Value': entry['value'],
        'Unit': entry['unit'],
        'Obs Status': entry['obs_status'],
        'Decimal': entry['decimal']
    }
    transformed_data.append(transformed_entry)

# Create a DataFrame from the transformed data
df = pd.DataFrame(transformed_data)

# Now you have a Pandas DataFrame that represents your dataset
# You can perform various data operations and analysis with it

In [ ]:
df = df[["Country ISO3 Code","Value"]]

In [ ]:
merged2016 = merged[(merged['year'] == 2016)]


merged2016 = merged2016.merge(df, left_on="country_code", right_on="Country ISO3 Code")


In [ ]:
per_country = merged2016.groupby(['country_code','medal' ]).size()

In [ ]:
per_country.index